In [25]:
!pip install pandas

In [26]:
!pip install pyarrow


In [51]:
!pip install fastparquet

  Using cached fsspec-2024.6.1-py3-none-any.whl.metadata (11 kB)
   ---------------------------------------- 0.0/672.1 kB ? eta -:--:--
   ---------------------------------------- 672.1/672.1 kB 5.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ---------------------------------------- 1.6/1.6 MB 14.6 MB/s eta 0:00:00
Using cached fsspec-2024.6.1-py3-none-any.whl (177 kB)


In [27]:
# Importar las librerías

import pandas as pd
import ast
import parquet

In [28]:
# Carga de datos de las películas y de los créditos

data_movies = pd.read_csv("C:/Users/cgabr/OneDrive/Documentos/Henry/Movies/API/Data/movies_dataset.csv")
data_credits = pd.read_csv("C:/Users/cgabr/OneDrive/Documentos/Henry/Movies/API\Data/credits.csv")

C:\Users\cgabr\AppData\Local\Temp\ipykernel_2584\687473288.py:3: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  data_movies = pd.read_csv("C:/Users/cgabr/OneDrive/Documentos/Henry/Movies/API/Data/movies_dataset.csv")


In [29]:
# Transformación a dataframes de las columnas belongs_to_collection y production_companies para desanidarlas

df_btc = pd.DataFrame(data=data_movies['belongs_to_collection'])
df_btc

df_prod = pd.DataFrame(data=data_movies['production_companies'])
df_prod

,production_companies
0,"[{'name': 'Pixar Animation Studios', 'id': 3}]"
1,"[{'name': 'TriStar Pictures', 'id': 559}, {'na..."
2,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'..."
3,[{'name': 'Twentieth Century Fox Film Corporat...
4,"[{'name': 'Sandollar Productions', 'id': 5842}..."
...,...
45461,[]
45462,"[{'name': 'Sine Olivia', 'id': 19653}]"
45463,"[{'name': 'American World Pictures', 'id': 6165}]"
45464,"[{'name': 'Yermoliev', 'id': 88753}]"


In [30]:
# Transformación a dataframe de la columna cast para desanidarlas

df_cast = pd.DataFrame(data=data_credits['cast'])
df_cast

,cast
0,"[{'cast_id': 14, 'character': 'Woody (voice)',..."
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '..."
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c..."
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah..."
4,"[{'cast_id': 1, 'character': 'George Banks', '..."
...,...
45471,"[{'cast_id': 0, 'character': '', 'credit_id': ..."
45472,"[{'cast_id': 1002, 'character': 'Sister Angela..."
45473,"[{'cast_id': 6, 'character': 'Emily Shaw', 'cr..."
45474,"[{'cast_id': 2, 'character': '', 'credit_id': ..."


In [31]:
df_btc = df_btc.drop_duplicates()
df_btc = df_btc.dropna()

In [32]:
df_prod = df_prod.drop_duplicates()
df_prod = df_prod.dropna()

In [33]:
df_cast = df_cast.drop_duplicates()
df_cast = df_cast.dropna()

In [34]:

# Conviersión de la columna "cast" a listas de diccionarios
data_credits['cast'] = data_credits['cast'].apply(ast.literal_eval)

# Aplica la función lambda para extraer los nombres de los actores
data_credits['actors'] = data_credits['cast'].apply(lambda x: [actor['name'] for actor in x])

# Verificación del resultado
print(data_credits['actors'])

0        [Tom Hanks, Tim Allen, Don Rickles, Jim Varney...
1        [Robin Williams, Jonathan Hyde, Kirsten Dunst,...
2        [Walter Matthau, Jack Lemmon, Ann-Margret, Sop...
3        [Whitney Houston, Angela Bassett, Loretta Devi...
4        [Steve Martin, Diane Keaton, Martin Short, Kim...
                               ...                        
45471          [Leila Hatami, Kourosh Tahami, Elham Korda]
45472    [Angel Aquino, Perry Dizon, Hazel Orencio, Joe...
45473    [Erika Eleniak, Adam Baldwin, Julie du Page, J...
45474    [Iwan Mosschuchin, Nathalie Lissenko, Pavel Pa...
45475                                                   []
Name: actors, Length: 45476, dtype: object


In [35]:
# Convierte la columna "crew" en listas de diccionarios utilizando la función ast.literal_eval
data_credits['crew'] = data_credits['crew'].apply(ast.literal_eval)

# Utiliza una función lambda junto con una comprensión de lista para filtrar los elementos en los que el valor de la clave "job" sea igual a "Director" y extraer el valor de la clave "name" para esos elementos.
data_credits['director_names'] = data_credits['crew'].apply(lambda x: [item['name'] for item in x if item['job'] == 'Director'])

# Convierte la lista de nombres de directores en un solo string separado por comas.
data_credits['director_names'] = data_credits['director_names'].apply(', '.join)
data_credits.head()

# Crea un nuevo dataframe con los datos de las columnas 
new_data_credits = data_credits.loc[:, ['id', 'actors', 'director_names']].copy()


In [36]:
# Se cambian los valores nulos por el valor 0

revenue = data_movies['revenue'].fillna(0)
budget = data_movies['budget'].fillna(0)

In [37]:
# Se eliminan los valores nulos de la columna release_date

data_movies.dropna(subset=['release_date'], inplace=True) 

In [38]:
# Se crea la columna release_year con los datos transformados a datetime

data_movies['release_year'] = pd.to_datetime(data_movies['release_date'], format='%Y-%m-%d', errors='coerce')

In [39]:
# Se convierten los datos a números para poder dividir las dos columnas

data_movies['revenue'] = pd.to_numeric(data_movies['revenue'], errors='coerce')
data_movies['budget'] = pd.to_numeric(data_movies['budget'], errors='coerce')

In [40]:
# Se divide la columna revenue sobre budget para crear la columna return con el cociente de lasdos anteriores

data_movies.loc[:, 'return'] = data_movies.apply(lambda row: row['revenue'] / row['budget'] if pd.notnull(row['revenue']) and pd.notnull(row['budget']) and row['budget'] != 0 else 0, axis=1)

data_movies['return']

0        12.451801
1         4.043035
2         0.000000
3         5.090760
4         0.000000
           ...    
45460     0.000000
45462     0.000000
45463     0.000000
45464     0.000000
45465     0.000000
Name: return, Length: 45379, dtype: float64

In [41]:
# Se eliminan las columnas que no se van a utilizar en el futuro

data_movies = data_movies.drop('video', axis=1)
data_movies = data_movies.drop('imdb_id', axis=1)
data_movies = data_movies.drop('adult', axis=1)
data_movies = data_movies.drop('original_title', axis=1)
data_movies = data_movies.drop('poster_path', axis=1)
data_movies = data_movies.drop('homepage', axis=1)

In [42]:
# Se corrige los valores de la columna popularity

data_movies['popularity'] = pd.to_numeric(data_movies['popularity'], errors='coerce')

In [48]:
data_credits = data_credits.drop('crew', axis=1)

In [59]:
# Finalmente se transforman los datos a parquet para disminuir su peso

data_movies.to_parquet('data_movies.parquet', engine='pyarrow')
data_credits.to_parquet('data_credits.parquet', engine='pyarrow')

ImportError: Missing optional dependency 'pyarrow'. pyarrow is required for parquet support. Use pip or conda to install pyarrow.

In [ ]:
data_movies.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [ ]:
data_movies['id'] = data_movies['id'].dropna()

In [ ]:
data_movies['id'] = pd.to_numeric(data_movies['id'], errors='coerce')
data_movies['popularity'] = pd.to_numeric(data_movies['popularity'], errors='coerce')